<a href="https://colab.research.google.com/github/Billwuzl/Domain-Adaptation/blob/main/Domain_Adaptation_New_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

# import needed lib
import numpy as np
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import json
import torch

from datasets import Dataset, DatasetDict
from datasets import load_metric, load_dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, fbeta_score, precision_score, recall_score,f1_score


import transformers
from transformers import AutoModel, BertTokenizer, BertForPreTraining
from transformers import (AutoTokenizer, AutoConfig,
                          AutoModelForSequenceClassification)
from transformers import Trainer,TrainingArguments

import glob
import random
import datetime
import dask.dataframe as dd



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [3]:
dataset = load_dataset("vai-2022/webpage_data",use_auth_token=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
import pandas as pd

In [5]:
df = pd.DataFrame(data=dataset['train']['0'])

In [6]:
df.rename(columns = {0: 'text',}, inplace = True)

In [7]:
df

,text
0,FREE shipping when you spend over £20. FREE sh...
1,The next generation of cloud computing. Data p...
2,We know what we’re making this weekend. This m...
3,Check you’ve filled out all the required field...
4,"Voted twice ""Best Beauty Products"" by the Fina..."
...,...
10085,Audience & Media Measurement. 7 ways logistics...
10086,Our therapists say. Our coaches say. Apps we l...
10087,Bringing innovative language solutions to the ...
10088,I am an asset owner:. I am a software company ...


In [8]:
df_train, df_unsup = train_test_split(df, shuffle=True,test_size=0.2)

# df_train = df_train.sample(n=10000,random_state=1)
ds= DatasetDict({
    "train": Dataset.from_pandas(df_train.reset_index(drop=True)),
    "unsup": Dataset.from_pandas(df_unsup.reset_index(drop=True))})

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True,
                     max_length=400, return_special_tokens_mask=True)

ds_mlm = ds.map(tokenize, batched=True)

ds_mlm = ds_mlm.remove_columns(["text"])

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling, set_seed

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir = f"{model_ckpt}-New_data_disbert1", 
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    logging_strategy="epoch", evaluation_strategy="epoch", save_strategy="no",
    num_train_epochs=16, push_to_hub=True, log_level="error", report_to="none")

trainer = Trainer(
        model=AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased"),
        tokenizer=tokenizer, args=training_args, data_collator=data_collator,
        train_dataset=ds_mlm["train"], eval_dataset=ds_mlm["unsup"])

trainer.train()

/content/distilbert-base-uncased-New_data_disbert1 is already a clone of https://huggingface.co/Billwzl/distilbert-base-uncased-New_data_disbert1. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub("Training complete!")

df_log = pd.DataFrame(trainer.state.log_history)

(df_log.dropna(subset=["eval_loss"]).reset_index()["eval_loss"]
 .plot(label="Validation"))
df_log.dropna(subset=["loss"]).reset_index()["loss"].plot(label="Train")

plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.show()